# Imports

In [1]:
import torch.nn as nn

In [2]:
# Conventional and convolutional neural network

class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [3]:
# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
    
        # Convolution layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        # Max pooling layer (divides image size by 2)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 500)
        self.fc2 = nn.Linear(500, 120)
        
        # Dropout
        self.dropout = nn.Dropout(0.3)
        
        
    def forward(self, x):
        ## Define forward behavior
        
        # Sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten image input
        x = x.view(-1, 128 * 28 * 28)
        # Dropout layer
        x = self.dropout(x)
        # 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # Dropout layer
        x = self.dropout(x)
        # 2nd hidden layer
        x = self.fc2(x)
        return x

In [4]:
class XV3V2N(nn.Module):
    
    def __init__(self):
        super(XV3V2N, self).__init__()
        
        self.base_model_1 = xception.Xception(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH,3))

        self.base_model_2 = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH,3))

        self.base_model_3 = inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH,3))

        # base_model_4 = resnet_v2.ResNet152V2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH,3))

        self.base_model_5 = nasnet.NASNetLarge(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH,3))

        # train only the top layers (which were randomly initialized)
        # i.e. freeze all convolutional Xception layers
        self.base_model_1.trainable = False
        self.base_model_2.trainable = False
        self.base_model_3.trainable = False
        # base_model_4.trainable = False
        self.base_model_5.trainable = False

    def forward(self, inputs):
        ## <-----  Xception   -----> ##
        x1 = xception.preprocess_input(inputs)
        # The base model contains batchnorm layers. We want to keep them in inference mode
        # when we unfreeze the base model for fine-tuning, so we make sure that the
        # base_model is running in inference mode here by passing `training=False`.
        x1 = self.base_model_1(x1, training=False)
        x1 = GlobalAveragePooling2D()(x1)

        ## <-----  InceptionV3   -----> ##
        x2 = inception_v3.preprocess_input(inputs)
        x2 = self.base_model_2(x2, training=False)
        x2 = GlobalAveragePooling2D()(x2)

        ## <-----  InceptionResNetV2   -----> ##
        x3 = inception_resnet_v2.preprocess_input(inputs)
        x3 = self.base_model_3(x3, training=False)
        x3 = GlobalAveragePooling2D()(x3)

        ## <-----  ResNet152V2   -----> ##
        # x4 = resnet_v2.preprocess_input(aug_inputs)
        # x4 = base_model_4(x4, training=False)
        # x4 = GlobalAveragePooling2D()(x4)

        ## <-----  NASNetLarge   -----> ##
        x5 = nasnet.preprocess_input(inputs)
        x5 = self.base_model_5(x5, training=False)
        x5 = GlobalAveragePooling2D()(x5)

        ## <-----  Concatenation  -----> ##
        x = Concatenate()([x1, x2, x3, x5])
        x = Dropout(.7)(x)
        outputs = Dense(120, activation='softmax')(x)
        return outputs